In [6]:
import cv2
import pandas as pd
import numpy as np
from ultralytics import YOLO
from tracker import *
import serial.tools.list_ports


ports = serial.tools.list_ports.comports()

serialInst = serial.Serial()

portsList = []

for one in ports:
    portsList.append(str(one))
    print(str(one))


com = input("Select comports for Arduino")


for i in range((len(portsList))):
    if portsList[i].startswith('COM'+str(com)):
        use = "COM" + str(com)
        print(use)

serialInst.baudrate = 9600
serialInst.port = use
serialInst.open()


model=YOLO('yolov8s.pt')



def RGB(event, x, y, flags, param):
    if event == cv2.EVENT_MOUSEMOVE :  
        colorsBGR = [x, y]
        print(colorsBGR)
        


cv2.namedWindow('RGB')
cv2.setMouseCallback('RGB',RGB)

cap=cv2.VideoCapture("Traffic IP Camera video (1).mp4")

my_file = open("coco.txt", "r")
data = my_file.read()   
class_list = data.split("\n")
print(class_list)
count=0
tracker = Tracker()

car_entering = {}
count_car = []
area = [(881,260),(508,333),(494,260),(846,212)]

def signal_counter(counted_car):
    if counted_car == 0:
        pass
    else:
        number_of_cars = counted_car
        mean_time_headway = 30/number_of_cars
        flow_rate = 3600/mean_time_headway
        Amber = 2
 

        approach_width = 4

        saturation_flow = 525*approach_width

        Y_ratio = saturation_flow/flow_rate


        critical_Y_ratio = Y_ratio

        number_of_phases = 2

        time_loss = 2

        all_red = 4

        loss_time_per_cycle = number_of_phases*time_loss + all_red


        Total_cycle_length = (1.5*loss_time_per_cycle + 5)/(1-critical_Y_ratio)

        green_signal_nb  = round(abs( Y_ratio/critical_Y_ratio *(Total_cycle_length-loss_time_per_cycle)))

        red_signal_nb = Total_cycle_length - green_signal_nb - Amber

        return green_signal_nb

while True:

    ret,frame = cap.read()
    if not ret:
        break
    count += 1
    if count % 3 != 0:
        continue
    frame=cv2.resize(frame,(1020,500))
    

    results=model.predict(frame)
    a=results[0].boxes.data
    px = pd.DataFrame(a).astype('float')
    list = []
    for index,row in px.iterrows():
        x1=int(row[0])
        y1=int(row[1])
        x2=int(row[2])
        y2=int(row[3])
        d=int(row[5])
        c=class_list[d]
        '''if 'car' in c:
            result = cv2.pointPolygonTest(np.array(area,np.int32),((x1,y2)),False)
            if result >= 0:
                cv2.rectangle(frame,(x1,y1),(x2,y2),(0,0,255),2)
                cv2.circle(frame,(x1,y2),4,(2555,0,0),-1)
                cv2.putText(frame,str(c),(x1,y1),cv2.FONT_HERSHEY_COMPLEX,(0.5),(255,255,255))'''
        if 'car' in c:
           list.append([x1,y1,x2,y2])
    bbox_id = tracker.update(list)
    for bbox in bbox_id:
        x3,y3,x4,y4,id = bbox
        result=cv2.pointPolygonTest(np.array(area,np.int32),((x4,y4)),False)
        if result >= 0:
            car_entering[id]=(x4,y4)
            cv2.rectangle(frame,(x3,y3),(x4,y4),(0,255,0),2)
            cv2.circle(frame,(x4,y4),5,(2555,0,0),-1)
            cv2.putText(frame,str(id),(x3,y3),cv2.FONT_HERSHEY_COMPLEX,(0.5),(255,255,255))
            if count_car.count(id)==0:
                count_car.append(id)
        
    cv2.polylines(frame,[np.array(area,np.int32)],True,(255,0,255),2)
    print(count_car)
    number = len(count_car) 
    timing = signal_counter(number)
    cv2.putText(frame,('Number of Cars: ')+ str(number),(60,40),cv2.FONT_HERSHEY_COMPLEX,0.8,(0,255,255),2)
    cv2.putText(frame,('Green Light =')+ str(signal_counter(number))+('s'),(60,80),cv2.FONT_HERSHEY_COMPLEX,0.8,(0,255,255),2)
    print('The number of cars from camera 1 is ' + str(len(count_car)))
    print('Green Light = ' + str(signal_counter(number)))    
    cv2.imshow("RGB", frame)
    command = str(timing)
    serialInst.write(command.encode('utf-8'))
    if cv2.waitKey(1)&0xFF== ord('q'):
        break

    

cap.release()
cv2.destroyAllWindows()


NameError: name 'use' is not defined

In [11]:
import cv2
import numpy as np

def detect_white(frame, min_area_threshold=500):
    hsv_image = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    # Define white color range in HSV
    lower_white1 = np.array([0, 100, 20], dtype=np.uint8)
    upper_white1 = np.array([10, 255, 255], dtype=np.uint8)

    lower_white2 = np.array([160, 100, 20], dtype=np.uint8)
    upper_white2 = np.array([179, 255, 255], dtype=np.uint8)

    lower_sugar = np.array([0, 0, 200], dtype=np.uint8)
    upper_sugar = np.array([255, 30, 255], dtype=np.uint8)

    lower_maggi = np.array([0, 0, 200], dtype=np.uint8)
    upper_maggi = np.array([0, 0, 200], dtype=np.uint8)

    # Create a mask for white color
    white_mask1 = cv2.inRange(hsv_image, lower_white1, upper_white1)
    white_mask2 = cv2.inRange(hsv_image, lower_white2, upper_white2)

    white_mask = white_mask1+white_mask2

    # Find contours in the mask
    contours, _ = cv2.findContours(
        white_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Iterate through the contours and filter based on area
    for contour in contours:
        area = cv2.contourArea(contour)
        if area > min_area_threshold:
            x, y, w, h = cv2.boundingRect(contour)
            confidence = (area / float(w * h)) * 100
            confidence_str = f"Red: {confidence:.2f}%"
            # Increased thickness
            frame = cv2.rectangle(
                frame, (x, y), (x + w, y + h), (255, 255, 255), 5)
            frame = cv2.putText(frame, confidence_str, (x, y - 10),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)

            # Check for rectangle or square
            if abs(w - h) <= 10:
                shape_str = "Square"
            else:
                shape_str = "Rectangle"

            frame = cv2.putText(frame, shape_str, (x + w // 2 - 30, y + h + 20),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)

    return frame,white_mask

#lower_white = np.array([0, 0, 200], dtype=np.uint8)
#upper_white = np.array([255, 30, 255], dtype=np.uint8)

cap = cv2.VideoCapture(0)



while True:
    ret,frame = cap.read()
    frame,mask = detect_white(frame, min_area_threshold=500)


    #lower_white , upper_white = get_limits(color = white)


    cv2.imshow('frame',frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()

cv2.destroyAllWindows()

In [1]:
#working code
import cv2
import pandas as pd
import numpy as np
from ultralytics import YOLO
from tracker import *
import serial.tools.list_ports
import time


ports = serial.tools.list_ports.comports()

serialInst = serial.Serial()

portsList = []

for one in ports:
    portsList.append(str(one))
    print(str(one))


com = input("Select comports for Arduino")


for i in range((len(portsList))):
    if portsList[i].startswith('COM'+str(com)):
        use = "COM" + str(com)
        print(use)

serialInst.baudrate = 9600
serialInst.port = use
serialInst.open()


model = YOLO('yolov8s.pt')


def RGB(event, x, y, flags, param):
    if event == cv2.EVENT_MOUSEMOVE:
        colorsBGR = [x, y]
        print(colorsBGR)


cv2.namedWindow('RGB')
cv2.setMouseCallback('RGB', RGB)

cap = cv2.VideoCapture(0)

my_file = open("coco.txt", "r")
data = my_file.read()
class_list = data.split("\n")
print(class_list)
count = 0
tracker = Tracker()

car_entering = {}
count_car = []
area = [(979,332),(605,270),(623,249),(986,301)]


def signal_counter(counted_car):
    if counted_car == 0:
        pass
    else:
        number_of_cars = counted_car
        mean_time_headway = 30 / number_of_cars
        flow_rate = 3600 / mean_time_headway
        Amber = 2

        approach_width = 4

        saturation_flow = 525 * approach_width

        Y_ratio = saturation_flow / flow_rate

        critical_Y_ratio = Y_ratio

        number_of_phases = 2

        time_loss = 2

        all_red = 4

        loss_time_per_cycle = number_of_phases * time_loss + all_red

        Total_cycle_length = (1.5 * loss_time_per_cycle + 5) / (1 - critical_Y_ratio)

        green_signal_nb = round(
            abs(Y_ratio / critical_Y_ratio * (Total_cycle_length - loss_time_per_cycle))
        )

        red_signal_nb = Total_cycle_length - green_signal_nb - Amber

        return green_signal_nb


def detect_white(frame, x1, y1, x2, y2, min_area_threshold=500):
    car_roi = frame[y1:y2, x1:x2]
    hsv_image = cv2.cvtColor(car_roi, cv2.COLOR_BGR2HSV)

    # Define white color range in HSV
    lower_white1 = np.array([0, 100, 20], dtype=np.uint8)
    upper_white1 = np.array([10, 255, 255], dtype=np.uint8)

    lower_white2 = np.array([160, 100, 20], dtype=np.uint8)
    upper_white2 = np.array([179, 255, 255], dtype=np.uint8)

    # Create a mask for white color
    white_mask1 = cv2.inRange(hsv_image, lower_white1, upper_white1)
    white_mask2 = cv2.inRange(hsv_image, lower_white2, upper_white2)

    white_mask = white_mask1+white_mask2

    # Find contours in the mask
    contours, _ = cv2.findContours(
        white_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE
    )

    # Iterate through the contours and filter based on area
    for contour in contours:
        area = cv2.contourArea(contour)
        if area > min_area_threshold:
            return True

    return False


while True:
    ret, frame = cap.read()
    if not ret:
        break
    count += 1
    if count % 10 != 0:
        continue
    frame = cv2.resize(frame, (1020, 500))

    results = model.predict(frame)
    a = results[0].boxes.data
    px = pd.DataFrame(a).astype('float')
    list = []
    emergency_vehicle_detected = False
    for index, row in px.iterrows():
        x1 = int(row[0])
        y1 = int(row[1])
        x2 = int(row[2])
        y2 = int(row[3])
        d = int(row[5])
        c = class_list[d]
        if 'car' in c:
            if detect_white(frame, x1, y1, x2, y2):
                emergency_vehicle_detected = True
                break
            else:
                list.append([x1, y1, x2, y2])

    if emergency_vehicle_detected:
        cv2.putText(frame, 'Emergency Vehicle Detected', (60, 40), cv2.FONT_HERSHEY_COMPLEX, 0.8, (0, 255, 255), 2)
        cv2.putText(frame, 'Green Light = 20s', (60, 80), cv2.FONT_HERSHEY_COMPLEX, 0.8, (0, 255, 255), 2)
        command = "T2-" + str(20)
        time.sleep(2)
        serialInst.write(command.encode('utf-8'))
    else:
        bbox_id = tracker.update(list)
        for bbox in bbox_id:
            x3, y3, x4, y4, id = bbox
            result = cv2.pointPolygonTest(np.array(area, np.int32), ((x4, y4)), False)
            if result >= 0:
                car_entering[id] = (x4, y4)
                cv2.rectangle(frame, (x3, y3), (x4, y4), (0, 255, 0), 2)
                cv2.circle(frame, (x4, y4), 5, (255, 0, 0), -1)
                cv2.putText(frame, str(id), (x3, y3), cv2.FONT_HERSHEY_COMPLEX, 0.5, (255, 255, 255))
                if count_car.count(id) == 0:
                    count_car.append(id)

        cv2.polylines(frame, [np.array(area, np.int32)], True, (255, 0, 255), 2)
        print(count_car)
        number = len(count_car)
        timing = signal_counter(number)
        cv2.putText(frame, 'Number of Cars: ' + str(number), (60, 40), cv2.FONT_HERSHEY_COMPLEX, 0.8, (0, 255, 255), 2)
        cv2.putText(frame, 'Green Light =' + str(signal_counter(number)) + 's', (60, 80), cv2.FONT_HERSHEY_COMPLEX, 0.8, (0, 255, 255), 2)
        print('The number of cars from camera 1 is ' + str(len(count_car)))
        print('Green Light = ' + str(signal_counter(number)))
    
    cv2.imshow("RGB", frame)
    command = "T2-" + str(timing)
    time.sleep(2)
    serialInst.write(command.encode('utf-8'))
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


COM16 - USB-SERIAL CH340 (COM16)
COM16
['person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed', 'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush']

0: 320x640 1 dog, 540.8ms
Speed: 26.4ms preprocess, 540.8ms inference, 2103.4ms postprocess per im

In [18]:
c = input("Arduino Command:")

type(c)

str

In [1]:
import serial.tools.list_ports
import time

ports = serial.tools.list_ports.comports()

serialInst = serial.Serial()

portsList = []

for one in ports:
    portsList.append(str(one))
    print(str(one))


com = input("Select comports for Arduino")


for i in range((len(portsList))):
    if portsList[i].startswith('COM'+str(com)):
        use = "COM" + str(com)
        print(use)

serialInst.baudrate = 9600
serialInst.port = use
serialInst.open()

while True:
    command = "T3-10"
    time.sleep(2)
    serialInst.write(command.encode('utf-8'))

    

COM17 - USB-SERIAL CH340 (COM17)
COM17


KeyboardInterrupt: 

In [29]:
import serial.tools.list_ports

# List available ports
ports = serial.tools.list_ports.comports()
serialInst = serial.Serial()

# Display and store the list of available ports
portsList = []
for one in ports:
    portsList.append(str(one))
    print(str(one))

# Predefined COM port and command
com = "16"  # Replace with the COM port number for your Arduino
command = "T1-7"  # Replace with the command you want to send to the Arduino

# Select the appropriate COM port
use = None
for i in range(len(portsList)):
    if portsList[i].startswith('COM' + str(com)):
        use = "COM" + str(com)
        print(use)
        break

if use:
    serialInst.baudrate = 9600
    serialInst.port = use
    serialInst.open()

    # Send the command to the Arduino
    serialInst.write(command.encode('utf-8'))

    # If you want to send commands repeatedly, you can uncomment the following loop
    # while True:
    #     serialInst.write(command.encode('utf-8'))
else:
    print(f"COM{com} not found in the available ports list.")


COM16 - USB-SERIAL CH340 (COM16)
COM16


In [1]:
import cv2
import pandas as pd
import numpy as np
from ultralytics import YOLO
from tracker import *
import serial.tools.list_ports

# Define the COM port for your Arduino
com_port = "COM16"  # Replace with the actual COM port number

# Set up serial communication
ports = serial.tools.list_ports.comports()
serialInst = serial.Serial()

# Display and store the list of available ports
portsList = [str(one) for one in ports]
for port in portsList:
    print(port)

# Check if the specified COM port is available
use = None
for port in portsList:
    if port.startswith(com_port):
        use = com_port
        print(f"Using {use}")
        break

if use:
    serialInst.baudrate = 9600
    serialInst.port = use
    serialInst.open()

    model = YOLO('yolov8s.pt')

    def RGB(event, x, y, flags, param):
        if event == cv2.EVENT_MOUSEMOVE:
            colorsBGR = [x, y]
            print(colorsBGR)

    cv2.namedWindow('RGB')
    cv2.setMouseCallback('RGB', RGB)

    cap = cv2.VideoCapture(0)

    with open("coco.txt", "r") as my_file:
        class_list = my_file.read().split("\n")
    print(class_list)

    count = 0
    tracker = Tracker()

    car_entering = {}
    count_car = []
    area = [(979, 332), (605, 270), (623, 249), (986, 301)]

    def signal_counter(counted_car):
        if counted_car == 0:
            return 0
        else:
            number_of_cars = counted_car
            mean_time_headway = 30 / number_of_cars
            flow_rate = 3600 / mean_time_headway
            Amber = 2

            approach_width = 4
            saturation_flow = 525 * approach_width
            Y_ratio = saturation_flow / flow_rate
            critical_Y_ratio = Y_ratio

            number_of_phases = 2
            time_loss = 2
            all_red = 4
            loss_time_per_cycle = number_of_phases * time_loss + all_red
            Total_cycle_length = (1.5 * loss_time_per_cycle + 5) / (1 - critical_Y_ratio)
            green_signal_nb = round(
                abs(Y_ratio / critical_Y_ratio * (Total_cycle_length - loss_time_per_cycle))
            )
            red_signal_nb = Total_cycle_length - green_signal_nb - Amber

            return green_signal_nb

    def detect_white(frame, x1, y1, x2, y2, min_area_threshold=500):
        car_roi = frame[y1:y2, x1:x2]
        hsv_image = cv2.cvtColor(car_roi, cv2.COLOR_BGR2HSV)

        # Define white color range in HSV
        lower_white1 = np.array([0, 100, 20], dtype=np.uint8)
        upper_white1 = np.array([10, 255, 255], dtype=np.uint8)

        lower_white2 = np.array([160, 100, 20], dtype=np.uint8)
        upper_white2 = np.array([179, 255, 255], dtype=np.uint8)

        # Create a mask for white color
        white_mask1 = cv2.inRange(hsv_image, lower_white1, upper_white1)
        white_mask2 = cv2.inRange(hsv_image, lower_white2, upper_white2)
        white_mask = white_mask1 + white_mask2

        # Find contours in the mask
        contours, _ = cv2.findContours(white_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        # Iterate through the contours and filter based on area
        for contour in contours:
            area = cv2.contourArea(contour)
            if area > min_area_threshold:
                return True

        return False

    while True:
        ret, frame = cap.read()
        if not ret:
            break
        count += 1
        if count % 10 != 0:
            continue
        frame = cv2.resize(frame, (1020, 500))

        results = model.predict(frame)
        a = results[0].boxes.data
        px = pd.DataFrame(a).astype('float')
        list = []
        emergency_vehicle_detected = False
        for index, row in px.iterrows():
            x1 = int(row[0])
            y1 = int(row[1])
            x2 = int(row[2])
            y2 = int(row[3])
            d = int(row[5])
            c = class_list[d]
            if 'car' in c:
                if detect_white(frame, x1, y1, x2, y2):
                    emergency_vehicle_detected = True
                    break
                else:
                    list.append([x1, y1, x2, y2])

        if emergency_vehicle_detected:
            cv2.putText(frame, 'Emergency Vehicle Detected', (60, 40), cv2.FONT_HERSHEY_COMPLEX, 0.8, (0, 255, 255), 2)
            cv2.putText(frame, 'Green Light = 20s', (60, 80), cv2.FONT_HERSHEY_COMPLEX, 0.8, (0, 255, 255), 2)
            command = "T1-" + str(20)
            serialInst.write(command.encode('utf-8'))
        else:
            bbox_id = tracker.update(list)
            for bbox in bbox_id:
                x3, y3, x4, y4, id = bbox
                result = cv2.pointPolygonTest(np.array(area, np.int32), ((x4, y4)), False)
                if result >= 0:
                    car_entering[id] = (x4, y4)
                    cv2.rectangle(frame, (x3, y3), (x4, y4), (0, 255, 0), 2)
                    cv2.circle(frame, (x4, y4), 5, (255, 0, 0), -1)
                    cv2.putText(frame, str(id), (x3, y3), cv2.FONT_HERSHEY_COMPLEX, 0.5, (255, 255, 255))
                    if count_car.count(id) == 0:
                        count_car.append(id)

            cv2.polylines(frame, [np.array(area, np.int32)], True, (255, 0, 255), 2)
            print(count_car)
            number = len(count_car)
            timing = signal_counter(number)
            cv2.putText(frame, 'Number of Cars: ' + str(number), (60, 40), cv2.FONT_HERSHEY_COMPLEX, 0.8, (0, 255, 255), 2)
            cv2.putText(frame, 'Green Light =' + str(timing) + 's', (60, 80), cv2.FONT_HERSHEY_COMPLEX, 0.8, (0, 255, 255), 2)
            print('The number of cars from camera 1 is ' + str(len(count_car)))
            print('Green Light = ' + str(timing))
            command = "T1-" + str(timing)
            serialInst.write(command.encode('utf-8'))

        cv2.imshow("RGB", frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

else:
    print(f"{com_port} not found in the available ports list.")


COM17 - USB-SERIAL CH340 (COM17)
COM16 not found in the available ports list.
